In [237]:
# import libraries
import pandas as pd
import numpy as np

In [238]:
# Load the dataset
data = pd.read_csv('C:\\Users\\Chase\\Downloads\\elden_ring_combat_weapons_raw.csv')
data.head()

,Name,Phy,Mag,Fire,Ligh,Holy,CritBoost,Str,Dex,Int,Fai,Arc,Wgt,Skill,Category,category_url,last_updated
0,Dagger,74 35,- 20,- 20,- 20,- 20,130 15,5 D,9 C,- -,- -,- -,1.5,3 Quickstep,Daggers,https://eldenring.wiki.fextralife.com/Daggers,15/08/2022
1,Parrying Dagger,75 35,- 20,- 20,- 20,- 20,110 15,5 E,14 C,- -,- -,- -,1.5,- Kick,Daggers,https://eldenring.wiki.fextralife.com/Daggers,15/08/2022
2,Misericorde,92 36,- 21,- 21,- 21,- 21,140 15,7 E,12 D,- -,- -,- -,2.0,3 Quickstep,Daggers,https://eldenring.wiki.fextralife.com/Daggers,15/08/2022
3,Great Knife,75 35,- 20,- 20,- 20,- 20,110 15,6 E,12 C,- -,- -,- -,1.5,3 Quickstep,Daggers,https://eldenring.wiki.fextralife.com/Daggers,15/08/2022
4,Bloodstained Dagger,81 36,- 21,- 21,- 21,- 21,110 15,9 C,12 E,- -,- -,- -,2.0,3 Quickstep,Daggers,https://eldenring.wiki.fextralife.com/Daggers,15/08/2022


In [239]:
# Split the dataset into two tables:
# Damage Stats: Name, Category, Phy, Mag, Fire, Ligh, Holy
# Level Requirements: Name, Str, Dex, Int, Fai, Arc
damage_stats = data[['Name', 'Category', 'Phy', 'Mag', 'Fire', 'Ligh', 'Holy']]
level_requirements = data[['Name', 'Category', 'Str', 'Dex', 'Int', 'Fai', 'Arc']]
display(damage_stats.head())
display(level_requirements.head())

,Name,Category,Phy,Mag,Fire,Ligh,Holy
0,Dagger,Daggers,74 35,- 20,- 20,- 20,- 20
1,Parrying Dagger,Daggers,75 35,- 20,- 20,- 20,- 20
2,Misericorde,Daggers,92 36,- 21,- 21,- 21,- 21
3,Great Knife,Daggers,75 35,- 20,- 20,- 20,- 20
4,Bloodstained Dagger,Daggers,81 36,- 21,- 21,- 21,- 21


,Name,Category,Str,Dex,Int,Fai,Arc
0,Dagger,Daggers,5 D,9 C,- -,- -,- -
1,Parrying Dagger,Daggers,5 E,14 C,- -,- -,- -
2,Misericorde,Daggers,7 E,12 D,- -,- -,- -
3,Great Knife,Daggers,6 E,12 C,- -,- -,- -
4,Bloodstained Dagger,Daggers,9 C,12 E,- -,- -,- -


In [240]:
# Pivot the two datasets:
# For Damage Stats, 1 column containing Phy, Mag, Fire, Ligh, Holy, and 1 for values.
# For Level Requirements, 1 column containing Str, Dex, Int, Fai, Arc, and 1 for vthe values
damage_stats = pd.melt(damage_stats, id_vars=['Name', 'Category'], value_vars=['Phy', 'Mag', 'Fire', 'Ligh', 'Holy'], var_name='Attribute', value_name='Score')
level_requirements = pd.melt(level_requirements, id_vars=['Name', 'Category'], value_vars=['Str', 'Dex', 'Int', 'Fai', 'Arc'], var_name='Attribute', value_name='Score')
display(damage_stats.head())
display(level_requirements.head())

,Name,Category,Attribute,Score
0,Dagger,Daggers,Phy,74 35
1,Parrying Dagger,Daggers,Phy,75 35
2,Misericorde,Daggers,Phy,92 36
3,Great Knife,Daggers,Phy,75 35
4,Bloodstained Dagger,Daggers,Phy,81 36


,Name,Category,Attribute,Score
0,Dagger,Daggers,Str,5 D
1,Parrying Dagger,Daggers,Str,5 E
2,Misericorde,Daggers,Str,7 E
3,Great Knife,Daggers,Str,6 E
4,Bloodstained Dagger,Daggers,Str,9 C


In [241]:
# Split the pivot values into two columns for both tables:
# For Damage Stats label the first column "Attack Damage", and the second "Damage Resistance" 
# For Level Requirements label the first column "Required Level", and the second "Attribute Scaling"
damage_stats[['Attack Damage', 'Damage Resistance']] = damage_stats['Score'].str.split(' ', expand=True)
level_requirements[['Required Level', 'Attribute Scaling']] = level_requirements['Score'].str.split(n=2, expand=True)
display(damage_stats.head())
display(level_requirements.head())

,Name,Category,Attribute,Score,Attack Damage,Damage Resistance
0,Dagger,Daggers,Phy,74 35,74,35
1,Parrying Dagger,Daggers,Phy,75 35,75,35
2,Misericorde,Daggers,Phy,92 36,92,36
3,Great Knife,Daggers,Phy,75 35,75,35
4,Bloodstained Dagger,Daggers,Phy,81 36,81,36


,Name,Category,Attribute,Score,Required Level,Attribute Scaling
0,Dagger,Daggers,Str,5 D,5,D
1,Parrying Dagger,Daggers,Str,5 E,5,E
2,Misericorde,Daggers,Str,7 E,7,E
3,Great Knife,Daggers,Str,6 E,6,E
4,Bloodstained Dagger,Daggers,Str,9 C,9,C


In [242]:
# Replace the "-" values with zero in the columns: "Attack Damage", "Damage Resistance", "Required Level", and change the data type to integers
damage_stats['Attack Damage'] = np.where(damage_stats['Attack Damage'] == "-", 0, damage_stats['Attack Damage'])
damage_stats['Damage Resistance'] = np.where(damage_stats['Damage Resistance'] == "-", 0, damage_stats['Damage Resistance'])
level_requirements['Required Level'] = np.where(level_requirements['Required Level'] == "-", 0, level_requirements['Required Level'])

# drop all unnecessary columns
damage_stats = damage_stats.drop(columns=['Score'])
level_requirements = level_requirements.drop(columns=['Score'])

# change the column types to integers
damage_stats['Attack Damage'] = damage_stats['Attack Damage'].astype(int)
damage_stats['Damage Resistance'] = damage_stats['Damage Resistance'].astype(int)
level_requirements['Required Level'] = level_requirements['Required Level'].astype(int)
display(damage_stats.head())
display(level_requirements.head())

,Name,Category,Attribute,Attack Damage,Damage Resistance
0,Dagger,Daggers,Phy,74,35
1,Parrying Dagger,Daggers,Phy,75,35
2,Misericorde,Daggers,Phy,92,36
3,Great Knife,Daggers,Phy,75,35
4,Bloodstained Dagger,Daggers,Phy,81,36


,Name,Category,Attribute,Required Level,Attribute Scaling
0,Dagger,Daggers,Str,5,D
1,Parrying Dagger,Daggers,Str,5,E
2,Misericorde,Daggers,Str,7,E
3,Great Knife,Daggers,Str,6,E
4,Bloodstained Dagger,Daggers,Str,9,C


In [243]:
# Find the total "Attack Damage" and total "Required Level" for all weapons and join the datasets together
damage_stats1 = damage_stats.groupby(['Name'])['Attack Damage'].sum()
damage_stats1 = pd.DataFrame(damage_stats1)
damage_stats1 = damage_stats1.reset_index()
damage_stats2 = level_requirements.groupby(['Name', 'Category'])['Required Level'].sum()
damage_stats2 = pd.DataFrame(damage_stats2)
damage_stats2 = damage_stats2.reset_index()
damage_stats = damage_stats1.merge(damage_stats2, how='inner', left_on='Name', right_on='Name')
damage_stats = damage_stats[['Name', 'Category', 'Attack Damage', 'Required Level']]
damage_stats

,Name,Category,Attack Damage,Required Level
0,Alabaster Lord's Sword,Greatswords,166,46
1,Antspur Rapier,Thrusting Swords,98,30
2,Axe of Godfrey,Colossal Weapons,165,56
3,Axe of Godrick,Greataxes,142,56
4,Bandit's Curved Sword,Curved Swords,118,24
...,...,...,...,...
251,Wing of Astel,Curved Swords,143,44
252,Winged Greathorn,Greataxes,130,50
253,Winged Scythe,Reapers,191,56
254,Zamor Curved Sword,Curved Greatswords,125,34


In [244]:
damage_stats['rank'] = damage_stats.groupby(['Required Level'])['Attack Damage'].rank(method='dense', ascending=False).astype(int)
damage_stats = damage_stats.loc[damage_stats['rank'] == 1, :]
damage_stats = damage_stats.drop(columns=['rank'])
damage_stats

,Name,Category,Attack Damage,Required Level
5,Banished Knight's Greatsword,Greatswords,142,26
11,Beast-Repellent Torch,Torches,162,20
12,Beastclaw Greathammer,Great Hammers,116,48
18,Bloodhound Claws,Claws,94,25
26,Caestus,Fists,90,16
28,Carian Knight's Sword,Straight Swords,176,38
36,Clayman's Harpoon,Spears,163,34
37,Claymore,Greatswords,138,29
52,Dagger,Daggers,74,14
56,Devourer's Scepter,Great Hammers,181,69
